<a href="https://colab.research.google.com/github/mantoan-thi/Exploratory-Analysis/blob/main/Modelagem_de_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycep-correios

In [179]:
import pandas as pd
import re
from pycep_correios import get_address_from_cep, WebService, exceptions
import time
import timeit

In [180]:
df = pd.read_csv('/content/GUIRRA E SANTANA clientes extraídos_7372.CSV', sep=';', encoding='latin-1')
# Removendo dados duplicados e deixando a primeira ocorrência da repetição 
df.drop_duplicates(keep='first', inplace=True) 

In [181]:
def verificar_cidade_bairro(x,y,z):
  if pd.isna(x)== False:
    try:
        address = get_address_from_cep(x, webservice=WebService.APICEP)
        cidade = str(address['cidade']).upper()
        bairro = str(address['logradouro']).upper()
        return cidade,bairro
        #return address
    except exceptions.InvalidCEP as eic:
        return y,z
    except exceptions.CEPNotFound as ecnf:
        return y,z
    except exceptions.ConnectionError as errc:
        return y,z
    except exceptions.Timeout as errt:
        return y,z
    except exceptions.HTTPError as errh:
        return y,z
    except exceptions.BaseException as e:
        return y,z
  else:
    return y,z

In [182]:
# Formatando CPF
def formatar_cpf(x):
  if pd.isna(x)== False:
    x = str(x).replace('.','')
    x = x.replace(' ','')
    x = x.replace(',','')
    x = x.replace('/','')
    x = x.replace('-','')
    x = x.replace("''",'')
    x = x.replace("'",'')
    x = x.replace("+",'')
    if len(x) == 11:
        x = x.zfill(11)
        cpf = '{}.{}.{}-{}'.format(x[:3], x[3:6], x[6:9], x[9:])
        return cpf
    elif len(x) == 10:
        x = str(0)+x
        x = x.zfill(11)
        cpf = '{}.{}.{}-{}'.format(x[:3], x[3:6], x[6:9], x[9:])
        return cpf
    else:
      return x
  else:
    return x

In [183]:
# Checar valores duplicados
def valore_duplicados(dados,coluna):
  valores = []
  repetidos = set()

  for dado in dados[coluna]:
      if dado not in valores:
          valores.append(dado)
      else:
          repetidos.add(dado)
  return valores,repetidos

In [184]:
def juntar_nome_cpf1(df,repetidos):
  for i in range(0,df.shape[0]):
    nome = df['PES.nome'].iloc[i]
    if nome in repetidos:
      for j in range(i+1,df.shape[0]):
        if str(df['PES.nome'].iloc[i]) == str(df['PES.nome'].iloc[j]) and str(df['PES.cpf_cnpj'].iloc[i]) != str(df['PES.cpf_cnpj'].iloc[j]):
          df['PES.nome'].iloc[j] = str(df['PES.nome'].iloc[j])+' '+str(df['PES.cpf_cnpj'].iloc[j])
        else:
          if str(df['PES.nome'].iloc[i]) == str(df['PES.nome'].iloc[j]) and str(df['PES.cpf_cnpj'].iloc[i]) == str(df['PES.cpf_cnpj'].iloc[j]):

            # Telefone
            if len(str(df['PES.telefones'].iloc[i]))>= len(str(df['PES.telefones'].iloc[j])):
                df['PES.telefones'].iloc[j] = df['PES.telefones'].iloc[i]
            if len(str(df['PES.telefones'].iloc[i])) <= len(str(df['PES.telefones'].iloc[j])):
                df['PES.telefones'].iloc[i] = df['PES.telefones'].iloc[j]

            if str(df['PES.telefones'].iloc[i]) == 'NaN':
                df['PES.telefones'].iloc[i] = df['PES.telefones'].iloc[j]
            else:
                df['PES.telefones'].iloc[j] = df['PES.telefones'].iloc[i]
            # Endereço
            if len(str(df['PES.endereco'].iloc[i]))>= len(str(df['PES.endereco'].iloc[j])):
                df['PES.endereco'].iloc[j] = df['PES.endereco'].iloc[i]
            if len(str(df['PES.endereco'].iloc[i])) <= len(str(df['PES.endereco'].iloc[j])):
                df['PES.endereco'].iloc[i] = df['PES.endereco'].iloc[j]
            
            # Bairro
            if len(str(df['PES.bairro'].iloc[i]))>= len(str(df['PES.bairro'].iloc[j])):
                df['PES.bairro'].iloc[j] = df['PES.bairro'].iloc[i]
            if len(str(df['PES.bairro'].iloc[i])) <= len(str(df['PES.bairro'].iloc[j])):
                df['PES.bairro'].iloc[i] = df['PES.bairro'].iloc[j]
            # cep
            if len(str(df['PES.cep'].iloc[i]))>= len(str(df['PES.cep'].iloc[j])):
                df['PES.cep'].iloc[j] = df['PES.cep'].iloc[i]
            if len(str(df['PES.cep'].iloc[i])) <= len(str(df['PES.cep'].iloc[j])):
                df['PES.cep'].iloc[i] = df['PES.cep'].iloc[j]

            # Informações
            if len(str(df['PES.outras_informacoes'].iloc[i]))>= len(str(df['PES.outras_informacoes'].iloc[j])):
                df['PES.outras_informacoes'].iloc[j] = df['PES.outras_informacoes'].iloc[i]
            if len(str(df['PES.outras_informacoes'].iloc[i])) <= len(str(df['PES.outras_informacoes'].iloc[j])):
                df['PES.outras_informacoes'].iloc[i] = df['PES.outras_informacoes'].iloc[j]

            # Cidade
            if len(str(df['PES.cidade'].iloc[i]))>= len(str(df['PES.cidade'].iloc[j])):
                df['PES.cidade'].iloc[j] = df['PES.cidade'].iloc[i]
            if len(str(df['PES.cidade'].iloc[i])) <= len(str(df['PES.cidade'].iloc[j])):
                df['PES.cidade'].iloc[i] = df['PES.cidade'].iloc[j]
            break
  df.drop_duplicates(keep='first', inplace=True) 
  return df

In [185]:
def juntar_nome_cpf2(df,repetidos):
  for i in range(0,df.shape[0]):
    nome = df['PES.cpf_cnpj'].iloc[i]
    if nome in repetidos:
      for j in range(0,df.shape[0]):
        if str(df['PES.nome'].iloc[i]) == str(df['PES.nome'].iloc[j]) and str(df['PES.cpf_cnpj'].iloc[i]) != str(df['PES.cpf_cnpj'].iloc[j]) and i!=j:
          df['PES.nome'].iloc[j] = str(df['PES.nome'].iloc[j])+' '+str(df['PES.cpf_cnpj'].iloc[j])

        elif str(df['PES.nome'].iloc[i]) != str(df['PES.nome'].iloc[j]) and str(df['PES.cpf_cnpj'].iloc[i]) == str(df['PES.cpf_cnpj'].iloc[j]):
          if len(str(df['PES.nome'].iloc[i])) >= len(str(df['PES.nome'].iloc[j])):
            df['PES.nome'].iloc[j] = str(df['PES.nome'].iloc[i])
          else:
              df['PES.nome'].iloc[i] = str(df['PES.nome'].iloc[j])

        else:
          if str(df['PES.nome'].iloc[i]) == str(df['PES.nome'].iloc[j]) and str(df['PES.cpf_cnpj'].iloc[i]) == str(df['PES.cpf_cnpj'].iloc[j]) and i!=j:

            # Telefone
            if len(str(df['PES.telefones'].iloc[i]))>= len(str(df['PES.telefones'].iloc[j])):
                df['PES.telefones'].iloc[j] = df['PES.telefones'].iloc[i]
            if len(str(df['PES.telefones'].iloc[i])) <= len(str(df['PES.telefones'].iloc[j])):
                df['PES.telefones'].iloc[i] = df['PES.telefones'].iloc[j]
            if pd.isna(str(df['PES.telefones'].iloc[i])):
                df['PES.telefones'].iloc[i] = df['PES.telefones'].iloc[j]
            else:
                df['PES.telefones'].iloc[j] = df['PES.telefones'].iloc[i]
            # Endereço
            if len(str(df['PES.endereco'].iloc[i]))>= len(str(df['PES.endereco'].iloc[j])):
                df['PES.endereco'].iloc[j] = df['PES.endereco'].iloc[i]
            if len(str(df['PES.endereco'].iloc[i])) <= len(str(df['PES.endereco'].iloc[j])):
                df['PES.endereco'].iloc[i] = df['PES.endereco'].iloc[j]
            
            # Bairro
            if len(str(df['PES.bairro'].iloc[i]))>= len(str(df['PES.bairro'].iloc[j])):
                df['PES.bairro'].iloc[j] = df['PES.bairro'].iloc[i]
            if len(str(df['PES.bairro'].iloc[i])) <= len(str(df['PES.bairro'].iloc[j])):
                df['PES.bairro'].iloc[i] = df['PES.bairro'].iloc[j]
            # cep
            if len(str(df['PES.cep'].iloc[i]))>= len(str(df['PES.cep'].iloc[j])):
                df['PES.cep'].iloc[j] = df['PES.cep'].iloc[i]
            if len(str(df['PES.cep'].iloc[i])) <= len(str(df['PES.cep'].iloc[j])):
                df['PES.cep'].iloc[i] = df['PES.cep'].iloc[j]

            # Informações
            if len(str(df['PES.outras_informacoes'].iloc[i]))>= len(str(df['PES.outras_informacoes'].iloc[j])):
                df['PES.outras_informacoes'].iloc[j] = df['PES.outras_informacoes'].iloc[i]
            if len(str(df['PES.outras_informacoes'].iloc[i])) <= len(str(df['PES.outras_informacoes'].iloc[j])):
                df['PES.outras_informacoes'].iloc[i] = df['PES.outras_informacoes'].iloc[j]

            # Cidade
            if len(str(df['PES.cidade'].iloc[i]))>= len(str(df['PES.cidade'].iloc[j])):
                df['PES.cidade'].iloc[j] = df['PES.cidade'].iloc[i]
            if len(str(df['PES.cidade'].iloc[i])) <= len(str(df['PES.cidade'].iloc[j])):
                df['PES.cidade'].iloc[i] = df['PES.cidade'].iloc[j]
            break
  df.drop_duplicates(keep='first', inplace=True) 
  return df

In [186]:
def formatar_tel1(x):
  if pd.isna(x)== False:
    cel = str(x.split('/')[0])
    cel = cel.replace('-','')
    cel = cel.replace(' ','')
    cel = cel.replace('.','')
    cel = cel.replace('(','')
    cel = cel.replace(')','')
    if len(cel)>=8:
      new_dig = ''
      for dig in cel:
        if dig.isdigit():
          new_dig = new_dig+dig
      cel =new_dig

    if len(cel)>= 8 and len(cel)<=12:
      cel = str(int(cel))
      return cel
    else:
      return x
  else:
    return x

In [187]:
def formatar_tel2(x):
  if pd.isna(x) == False:
    cel = str(x.split('-')[0])
    cel = cel.replace('-','')
    cel = cel.replace(' ','')
    cel = cel.replace('.','')
    cel = cel.replace('(','')
    cel = cel.replace(')','')
    if len(cel)>=8:
      new_dig = ''
      for dig in cel:
        if dig.isdigit():
          new_dig = new_dig+dig
      cel =new_dig

    if len(cel)>= 8 and len(cel)<=12:
      cel = str(int(cel))
      return cel
    else:
      return x
  else:
    return x

In [188]:
def formatar_tel3(x):
  if pd.isna(x) == False:
    cel = str(x.split('ou')[0])
    cel = cel.replace('-','')
    cel = cel.replace(' ','')
    cel = cel.replace('.','')
    cel = cel.replace('(','')
    cel = cel.replace(')','')
    if len(cel)>=8:
      new_dig = ''
      for dig in cel:
        if dig.isdigit():
          new_dig = new_dig+dig
      cel =new_dig

    if len(cel)>= 8 and len(cel)<=12:
      cel = str(int(cel))
      return cel
    else:
      return x
  else:
    return x

In [189]:
def formatar_tel4(x):
  if pd.isna(x) == False:
    cel = str(x.split(' - ')[0])
    cel = cel.replace('-','')
    cel = cel.replace(' ','')
    cel = cel.replace('.','')
    cel = cel.replace('(','')
    cel = cel.replace(')','')
    if len(cel)>=8:
      new_dig = ''
      for dig in cel:
        if dig.isdigit():
          new_dig = new_dig+dig
      cel =new_dig

    if len(cel)>= 8 and len(cel)<=12:
      cel = str(int(cel))
      return cel
    else:
      return x
  else:
    return x

In [190]:
def formatar_tel5(x):
  if pd.isna(x) == False:
    cel = str(x.split(' ')[0])
    cel = cel.replace('-','')
    cel = cel.replace(' ','')
    cel = cel.replace('.','')
    cel = cel.replace('(','')
    cel = cel.replace(')','')
    if len(cel)>=8:
      new_dig = ''
      for dig in cel:
        if dig.isdigit():
          new_dig = new_dig+dig
      cel =new_dig

    if len(cel)>= 8 and len(cel)<=12:
      cel = str(int(cel))
      return cel
    else:
      return x
  else:
    return x

In [191]:
def formatar_tel5(x):
    if pd.isna(x) == False:
      if len(str(x))==8:
        return '(00) {}-{}'.format(x[:4],x[4:])
      elif len(str(x))==9:
        return '(00) {}-{}'.format(x[:5],x[5:])
      elif len(str(x))==10:
        return '({}) {}-{}'.format(x[:2],x[2:6],x[6:])
      elif len(str(x))==11:
        return '({}) {}-{}'.format(x[:2],x[2:7],x[7:])
      return x
    else:
      return x

In [192]:
def remove_ultimo_espaco(x):
  if pd.isna(x) == False:
    new_x = " ".join(x.split())
    return re.sub(r"^\s+|\s+$", "", new_x)
  else:
    return x

In [193]:
def format_cep(x):
  if pd.isna(x) == False:
    cep = str(x).replace('-','')
    cep = cep.replace(' ','')
    cep = cep.replace('.','')
    cep = cep.replace('(','')
    cep = cep.replace(')','')
    if len(str(cep))==5:
      return '{}-000'.format(cep[:5])
    elif len(str(cep))==6:
      return '{}-{}00'.format(cep[:5],cep[5:6])
    elif len(str(cep))==7:
      return '{}-{}0'.format(cep[:5],cep[5:7])
    elif len(str(cep))==8:
      return '{}-{}'.format(cep[:5],cep[5:])
    else:
      return None
  else:
    return x

In [194]:
def alguma_funcao():
    for i in range(1, 5):
        time.sleep(1)

In [ ]:
inicio = timeit.default_timer()
print('Qtde de linhas antes do processamento:',df.shape[0])


# Manter somente linhas com pelo menos 2 valores que não sejam nan
df.dropna(thresh=2,inplace=True)

# Tratamento de Cep
df['PES.cep'] = df['PES.cep'].apply(lambda x:format_cep(x))
df.drop_duplicates(keep='first', inplace=True) 

# Tratamento de Bairro/cidade
try:
  df['PES.cidade'],df['PES.bairro'] = df.apply(lambda x:verificar_cidade_bairro(x['PES.cep'],x['PES.cidade'],x['PES.bairro']),axis=1)
except:
  print("An exception occurred")

df.drop_duplicates(keep='first', inplace=True) 


for i in range(0,2):

  # Remove os espaços no final do nome
  for c in df.columns:
    df[c] = df[c].apply(lambda x:remove_ultimo_espaco(x))
  df['PES.cpf_cnpj'] = df['PES.cpf_cnpj'].apply(lambda x:formatar_cpf(x))

  # Tratamento dos números de telefones
  df['PES.telefones'] = df['PES.telefones'].apply(lambda x:formatar_tel1(x))
  df['PES.telefones'] = df['PES.telefones'].apply(lambda x:formatar_tel2(x))
  df['PES.telefones'] = df['PES.telefones'].apply(lambda x:formatar_tel3(x))
  df['PES.telefones'] = df['PES.telefones'].apply(lambda x:formatar_tel4(x))
  df['PES.telefones'] = df['PES.telefones'].apply(lambda x:formatar_tel5(x))
  df['PES.telefones'] = df['PES.telefones'].apply(lambda x:formatar_tel5(x))
  df.drop_duplicates(keep='first', inplace=True) 

  # Tratamento dos números de CPF
  df['PES.cpf_cnpj'] = df['PES.cpf_cnpj'].apply(lambda x:formatar_cpf(x))

  # equiparando os dados
  v_,repetidos = valore_duplicados(df,'PES.nome')
  df = juntar_nome_cpf1(df,repetidos)


  _repetidos = valore_duplicados(df,'PES.cpf_cnpj')
  df = juntar_nome_cpf2(df,repetidos)
print('Qtde de linhas após o processamento:',df.shape[0])

print('\n\nVerificando valores duplicados por coluna\n')
for c in df.columns:
  _,repetidos = valore_duplicados(df,c)
  print(f'Valores repetidos em {c} ',len(repetidos))

fim = timeit.default_timer()
print ('Duracao: %f' % (fim - inicio))

Qtde de linhas antes do processamento: 9993


In [ ]:
df